In [1]:
import torch
from torchvision import models


/home/ning/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!pwd

/home/ning/Repo/kubeml/myExperiments


In [3]:
model0 = models.resnet18(pretrained= False)
model1 = models.resnet18(pretrained= False)

model0.load_state_dict(torch.load(f"./model_0.pt"))
model1.load_state_dict(torch.load(f"./model_1.pt"))

print(dir(model0)) 

['__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_buffers', '_forward_hooks', '_forward_impl', '_forward_pre_hooks', '_get_name', '_load_from_state_dict', '_load_state_dict_pre_hooks', '_make_layer', '_modules', '_named_members', '_norm_layer', '_parameters', '_register_load_state_dict_pre_hook', '_register_state_dict_hook', '_replicate_for_data_parallel', '_save_to_state_dict', '_slow_forward', '_state_dict_hooks', '_version', 'add_module', 'apply', 'avgpool', 'base_width', 'bn1', 'buffers', 'children', 'conv1', 'cpu', 'cuda', 'dilation', 'double', 'dump_patches', 'eval', 'extra_repr', 'fc', 'float', 'forward', 'groups', '

In [14]:
parallelism = 2
model = models.resnet18(pretrained= False)
cur_model = models.resnet18(pretrained= False)

beta = 1.0/parallelism 
for i in range(parallelism):
    cur_model.load_state_dict(torch.load(f"./model_{i}.pt"))
    for key in cur_model.state_dict():
      if i == 0:
        model.state_dict()[key] = (cur_model.state_dict()[key]) / parallelism
      else:
        model.state_dict()[key] += (cur_model.state_dict()[key]) / parallelism
    #print(model)


print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [32]:
import torch.nn as nn

# Setup
modelA = nn.Linear(3, 2)
modelB = nn.Linear(3, 2)
modelC = nn.Linear(3, 2)

print(modelA.state_dict())
print(modelB.state_dict())

sdA = modelA.state_dict()
sdB = modelB.state_dict()
sdC = modelC.state_dict()
# Average all parameters
for key in sdA:
    sdC[key] = (sdB[key] + sdA[key]) / 2.

# Recreate model and load averaged state_dict (or use modelA/B)
model = nn.Linear(3, 2)
model.load_state_dict(sdC)

print(model.state_dict())


for key in sdA:
    print(model.state_dict()[key] - (sdB[key] + sdA[key]) / 2)

OrderedDict([('weight', tensor([[ 0.3981,  0.3661, -0.4000],
        [-0.4682, -0.4987,  0.1842]])), ('bias', tensor([0.5153, 0.0663]))])
OrderedDict([('weight', tensor([[ 0.2693,  0.1602, -0.1152],
        [-0.4010,  0.0366, -0.1606]])), ('bias', tensor([-0.4497,  0.2997]))])
OrderedDict([('weight', tensor([[ 0.3337,  0.2631, -0.2576],
        [-0.4346, -0.2311,  0.0118]])), ('bias', tensor([0.0328, 0.1830]))])
tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


In [18]:
import torch
import torch.nn as nn


modelA = nn.Linear(3, 2)
modelB = nn.Linear(3, 2)
modelC = nn.Linear(3, 2)


torch.save(modelA.state_dict(), f"model_0.pt")
torch.save(modelB.state_dict(), f"model_1.pt")
torch.save(modelC.state_dict(), f"model_2.pt")

sdA = modelA.state_dict()
sdB = modelB.state_dict()
sdC = modelC.state_dict()

sd_avg = modelB.state_dict()
parallelism = 3
model = nn.Linear(3, 2)
cur_model = nn.Linear(3, 2)

for i in range(parallelism):
    cur_model.load_state_dict(torch.load(f"./model_{i}.pt"))
    for key in cur_model.state_dict():
      if i == 0:
        sd_avg[key] = ((cur_model.state_dict()[key]) / parallelism)
      else:
        sd_avg[key] += (cur_model.state_dict()[key]) / parallelism
    #print(model)

model.load_state_dict(sd_avg)
for key in sdA:
    print(model.state_dict()[key] - (sdB[key] + sdA[key] + sdC[key]) / parallelism)

tensor([[-7.4506e-09,  3.7253e-09,  0.0000e+00],
        [ 0.0000e+00, -7.4506e-09,  0.0000e+00]])
tensor([ 4.6566e-09, -2.9802e-08])
